In [ ]:
!pip install tf2onnx

In [5]:
import tensorflow as tf
import tf2onnx
import onnx

import boto3
from botocore.client import Config
from botocore.exceptions import ClientError
import os

In [ ]:
input_signature = [tf.TensorSpec([1, 256, 256, 3], tf.float32, name='x')]
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature, opset=13)

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
model_name = f"classification_model_{timestamp}.onnx"

onnx.save(onnx_model, model_name)

In [ ]:
# MinIO connection details
minio_endpoint = os.getenv('AWS_S3_ENDPOINT')
access_key = os.getenv('AWS_ACCESS_KEY_ID')
secret_key = os.getenv('AWS_SECRET_ACCESS_KEY')
bucket_name = os.getenv('AWS_S3_BUCKET')
region = os.getenv('AWS_DEFAULT_REGION')

# Initialize the S3 client
s3_client = boto3.client(
    's3',
    endpoint_url=minio_endpoint,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    config=Config(signature_version='s3v4')
)

In [11]:
try:
    response = s3_client.upload_file(model_name, 'models', 'fracture')
except ClientError as e:
    logging.error(e)

S3UploadFailedError: Failed to upload fracture_model.onnx to models/models: An error occurred (XMinioStorageFull) when calling the CreateMultipartUpload operation: Storage backend has reached its minimum free drive threshold. Please delete a few objects to proceed.